In [3]:
import tensorflow as tf
import sys
import subprocess
import json


2025-10-20 08:52:54.872337: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 08:52:55.379221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 08:52:56.897945: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
print("Python:", sys.version.splitlines()[0])
print()

Python: 3.11.14 | packaged by conda-forge | (main, Oct 13 2025, 14:09:32) [GCC 14.3.0]



In [6]:
try:
    import tensorflow as tf
    print("TensorFlow:", tf.__version__)
    gpus = tf.config.list_physical_devices('GPU')
    print("  GPUs detectadas por TensorFlow:", len(gpus))
    for i, g in enumerate(gpus):
        print(f"    [{i}] {g}")
    try:
        print("  TF built with CUDA:", tf.test.is_built_with_cuda())
    except Exception:
        # fallback: algumas versões não têm is_built_with_cuda
        print("  TF built with CUDA: (não disponível nesta versão do TF)")
except Exception as e:
    print("TensorFlow: não disponível ou erro ao importar:", e)

print()


build = tf.sysconfig.get_build_info()
print("Versão CUDA:", build.get("cuda_version", "desconhecida"))
print("Versão cuDNN:", build.get("cudnn_version", "desconhecida"))

TensorFlow: 2.20.0
  GPUs detectadas por TensorFlow: 1
    [0] PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
  TF built with CUDA: True

Versão CUDA: 12.5.1
Versão cuDNN: 9


In [1]:
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("  torch.cuda.is_available():", torch.cuda.is_available())
    try:
        count = torch.cuda.device_count()
        print("  GPUs detectadas por PyTorch:", count)
        for i in range(count):
            print(f"    [{i}] {torch.cuda.get_device_name(i)}")
            mem = torch.cuda.get_device_properties(i).total_memory
            print(f"      Memória total (bytes): {mem}")
    except Exception as e:
        print("  Erro ao obter detalhes da GPU via PyTorch:", e)
    print("  CUDA (compilado):", torch.version.cuda)
except Exception as e:
    print("PyTorch: não disponível ou erro ao importar:", e)

print()


PyTorch: 2.7.1
  torch.cuda.is_available(): True
  GPUs detectadas por PyTorch: 1
    [0] NVIDIA GeForce RTX 4060 Laptop GPU
      Memória total (bytes): 8585216000
  CUDA (compilado): 12.9



In [7]:
def run_cmd(cmd):
    try:
        out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
        return out.decode('utf-8', errors='ignore')
    except subprocess.CalledProcessError as e:
        return None

nvsmi = run_cmd("nvidia-smi -L")
if nvsmi:
    print("nvidia-smi -L (GPUs detectadas):")
    print(nvsmi.strip())
    print("nvidia-smi (sumário):")
    print(run_cmd("nvidia-smi --query-gpu=name,index,memory.total,driver_version --format=csv"))
else:
    print("nvidia-smi não encontrado ou não respondeu. Pode não haver driver NVIDIA instalado ou o utilitário não está no PATH.")
    # tentar informações do kernel
    mod = run_cmd("lsmod | grep nvidia || true")
    print("Módulos nvidia carregados (lsmod | grep nvidia):")
    print(mod or "(nenhum)")

nvidia-smi -L (GPUs detectadas):
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU (UUID: GPU-4cee1ad4-4832-56fc-971f-b46a3bae8f51)
nvidia-smi (sumário):
name, index, memory.total [MiB], driver_version
NVIDIA GeForce RTX 4060 Laptop GPU, 0, 8188 MiB, 581.57



In [3]:
import torch

# cria device explicitamente (assumindo que cuda está disponível)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Usando device:", device)


Usando device: cuda:0


In [ ]:

# exemplo: mover modelo e batch
model = MyModel()
model.to(device)                   # move pesos para GPU 0

# quando receber batch
images = images.to(device, non_blocking=True)
outputs = model(images)

In [ ]:
# Célula: verificação completa de GPU (TensorFlow, PyTorch, nvidia-smi)
import sys
import subprocess
import textwrap
from datetime import datetime

def hr(title=""):
    print("\n" + "="*80)
    if title:
        print(title)
        print("-"*len(title))

def run_cmd(cmd):
    try:
        out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT, timeout=10)
        return out.decode("utf-8", errors="ignore").strip()
    except Exception as e:
        return None

def human_bytes(n):
    # formata bytes em GB/MB
    for unit in ['B','KB','MB','GB','TB']:
        if abs(n) < 1024.0:
            return f"{n:3.1f}{unit}"
        n /= 1024.0
    return f"{n:.1f}PB"

print(f"Relatório gerado em: {datetime.now().isoformat(sep=' ', timespec='seconds')}")
hr("Python / Ambiente")
print("Python:", sys.version.splitlines()[0])

# TensorFlow
hr("TensorFlow")
try:
    import tensorflow as tf
    print("tensorflow:", tf.__version__)
    try:
        gpus = tf.config.list_physical_devices('GPU')
        print("  GPUs detectadas por TensorFlow:", len(gpus))
        for i, g in enumerate(gpus):
            print(f"    [{i}] {g}")
    except Exception as e:
        print("  Erro ao listar GPUs no TF:", e)
    try:
        built_cuda = tf.test.is_built_with_cuda()
        print("  TF built with CUDA:", built_cuda)
    except Exception:
        # fallback se função não existir
        build = tf.sysconfig.get_build_info()
        print("  TF build info (cuda/cudnn):",
              f"cuda={build.get('cuda_version','?')}, cudnn={build.get('cudnn_version','?')}")
except Exception as e:
    print("TensorFlow não disponível / erro ao importar:", str(e))

# PyTorch
hr("PyTorch")
try:
    import torch
    print("torch:", torch.__version__)
    cuda_avail = torch.cuda.is_available()
    print("  torch.cuda.is_available():", cuda_avail)
    try:
        dev_count = torch.cuda.device_count()
        print("  GPUs detectadas por PyTorch:", dev_count)
        for i in range(dev_count):
            try:
                name = torch.cuda.get_device_name(i)
            except Exception:
                name = "(nome indisponível)"
            try:
                props = torch.cuda.get_device_properties(i)
                mem = props.total_memory
            except Exception:
                mem = None
            print(f"    [{i}] {name}")
            if mem:
                print(f"       Memória total: {human_bytes(mem)}")
    except Exception as e:
        print("  Erro ao obter detalhes via PyTorch:", e)
    try:
        print("  CUDA (compilado):", torch.version.cuda)
    except Exception:
        pass
except Exception as e:
    print("PyTorch não disponível / erro ao importar:", str(e))

# nvidia-smi / driver
hr("nvidia-smi / Driver")
nvsmi_path = run_cmd("which nvidia-smi")
if nvsmi_path:
    print("nvidia-smi encontrado em:", nvsmi_path)
    print("\nSaída: nvidia-smi -L")
    print(run_cmd("nvidia-smi -L") or "(vazio)")
    print("\nSumário (name,index,memory.total,driver_version):")
    print(run_cmd("nvidia-smi --query-gpu=name,index,memory.total,driver_version --format=csv") or "(não retornou)")
    print("\nUtilização / status (top):")
    print(run_cmd("nvidia-smi --query-gpu=index,name,temperature.gpu,utilization.gpu,utilization.memory --format=csv,nounits") or "(não retornou)")
else:
    print("nvidia-smi não encontrado no PATH. Pode não haver driver NVIDIA instalado ou utilitário não está disponível.")
    # tenta checar módulos do kernel
    mod = run_cmd("lsmod | grep -i nvidia || true")
    print("\nMódulos nvidia carregados (lsmod | grep -i nvidia):")
    print(mod or "(nenhum)")

# Dicas rápidas
hr("Dicas rápidas / interpretação")
print(textwrap.dedent("""
- Se 'nvidia-smi' não existir: driver NVIDIA ausente; instale driver apropriado para sua distribuição.
- Se 'nvidia-smi' mostrar GPUs, mas torch.cuda.is_available() for False:
  - possivelmente PyTorch foi instalado sem suporte CUDA ou há incompatibilidade entre driver e runtime CUDA.
  - solução rápida: usar conda e instalar pytorch + pytorch-cuda compatível com sua versão de driver.
- Se TensorFlow não vê GPU: verifique se instalou a versão GPU (ou compatível) do TensorFlow e a compatibilidade CUDA/cuDNN.
- Em Docker: rode com '--gpus all' e instale 'nvidia-docker2' / 'nvidia-container-toolkit'.
- Se quiser, cole a saída completa desta célula aqui que eu indico os comandos exatos para instalar drivers / bibliotecas.
"""))
hr("Fim do relatório")